### Imports and environment

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

%load_ext autoreload
%autoreload 2

In [ ]:
from requests_toolbelt import MultipartEncoder
from requests_toolbelt.multipart import decoder
import requests
import os
import json
from PIL import Image
import io

In [ ]:
sample_input_root = os.getcwd()
sample_input_root = sample_input_root.replace('\\api_flask_redis\\api_core\\tests', '\\api\\synchronous\\sample_input')

ip_address = '100.100.200.200'
port = '5050'
base_url = 'http://{}:{}/v1/camera-trap/sync/'.format(ip_address, port)  # insert the IP address of the API
base_url

# Test

In [ ]:
sample_input_dir = os.path.join(sample_input_root, 'test_images')

In [ ]:
params = {
    'min_confidence': 0.15,
    'min_rendering_confidence': 0.8,
    'render': True
}

files = {}
sample_input_dir = os.path.join(sample_input_root, 'png')

for i, image_name in enumerate(sorted(os.listdir(sample_input_dir))):
    if image_name.startswith('.'):
        continue
    print(image_name)
    img_path = os.path.join(sample_input_dir, image_name)
    files[image_name] = (image_name, open(img_path, 'rb'), 'image/jpeg')
    
print('Number of files:', len(files))

m = MultipartEncoder(fields=files)
print(m.content_type)

r = requests.post(base_url + 'detect', 
                  params=params,
                  data=m,
                  headers={'Content-Type': m.content_type})
r.status_code
r.ok
if not r.ok:
    r.reason
    r.text
r.elapsed.total_seconds()

In [ ]:
results = decoder.MultipartDecoder.from_response(r)
text_results = {}
images = {}
for part in results.parts:
    # part is a BodyPart object with b'Content-Type', and b'Content-Disposition', the later includes 'name' and 'filename' info
    headers = {}
    for k, v in part.headers.items():
        headers[k.decode(part.encoding)] = v.decode(part.encoding)
    if headers.get('Content-Type', None) == 'image/jpeg':
        #images[part.headers['filename']] = part.content
        c = headers.get('Content-Disposition')
        image_name = c.split('name="')[1].split('"')[0]  # somehow all the filename and name info is all in one string with no obvious forma
        image = Image.open(io.BytesIO(part.content))
        
        images[image_name] = image
    
    elif headers.get('Content-Type', None) == 'application/json':
        text_result = json.loads(part.content.decode())
text_result

In [ ]:
for img_name, img in sorted(images.items()):
    print(img_name)
    img
    print()